<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C2-Writing_A_Sniffer/Packet_Sniffing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Writing A Sniffer__*

Network sniffers allow you to see packets entering and exiting a target machine. As a result, they have many practical uses before and after exploitation. In some cases, you'll be able to use existing sniffing tools like [Wireshark](https://wireshark.org/) or a Pythonic solution like __Scapy__ (which we'll explore in the next chapter). Nevertheless, there's an advantage to knowing how to throw together your own quick sniffer to view and decode network traffic.

Writing a tool like this will also give you a deep appreciation for the mature tools, as these can painlessly take care of the finer points with little effort on your part. You'll also likely pick up some new Pyhon techniques and perhaps a better understanding of how the low-level networking bits work.

In the previous chapter, we covered how to send and receive data using __TCP__ and __UDP__. This is likely how you'll interact with most network services.
But underneath these higher-level protocols are the building blocks that determine how network packets are sent and receiveid. You'll use raw sockets to access lower-level networking information, such as the raw Internet Protocol (__IP__) and Internet Control Message Protocol (__ICMP__) headers.

We won't decode any Ethernet information in this chapter, but if you intend to perform any low-level attacks, such as __ARP__ poisoning, or are developing wireless assessment tools, you should become intimately familiar with Ethernet frames and their use. Let's begin with a brief walk-through of how to discover active hosts on a network segment.

### *__Building an UDP Host Discovery Tool__*

Our sniffer's main goal is to discover hosts on a target network. Attackers want to be able to see all of the potential targets on a network so that they can focus their recoinnaissance and exploitation attempts.

We'll use a known behavior of most operating systems to determine if there is an active host at a particular __IP__ address. When we send an __UDP__ datagram to a closed port on a host, that host typically sends back an __ICMP__ message indicating that the port is unreachable. This __ICMP__ message tells us that there is an host alive, because if there was no host, we probably wouldn't receive any response to the __UDP__ datagram. It's essential, therefore, that we pick an __UDP__ port that won't likely be used. For maximum coverage, we can probe several ports to ensure we aren't hitting an active __UDP__ service.

Why the User Datagram Protocol? Well, there's no overhead in spraying the message across an entire subnet and waiting for the __ICMP__ responses to arrive accordingly. This is quite a simple scanner to build, as most of the work goes into decoding and analyzing the various network protocol headers. We'll implement this host scanner for both Winzozz and Linux to maximize the likelihood of being able to use it inside an enterprise environment.

We could also build additional logic into our scanner to kick off full __Nmap__ port scans on any host we discover. That way, we can determine if they have a viable network attack surface. This is an exercise left for the reader, and we the authors look foward to hearing some of the creative ways you can expand this core concept. Let's get started.

### *__Packet Sniffing on Winzozz and Linux__*

The process of accessing raw sockets in Winzozz is slightly different than on its Linux brethren, but we want the flexibility to deploy the same sniffer to multiple platforms. To account for this, we'll create a socket object and then determine which platform we're running on. 

Winzozz requires us to set some additional flags through a socket input/output control (__IOCTL__), which enables promiscuous mode on the network interface. An Input/Output Control (__IOCTL__) is a means for user space programs to communicate with kernel model components. Have a read [here](http://en.wikipedia.org/wiki/Ioctl).

In our first example, we simply set up our raw socket sniffer, read in a single packet, and then quit: